In [ ]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
from matplotlib_venn import venn2
import seaborn as sns

sns.set_style()
plt.rcParams["figure.figsize"] = (12, 6)

In [ ]:
# Lee egresos hospitalarios y consultas ambulatorias
df_egresos = pd.read_csv("../data/processed/df_procesada.csv")
df_consultas = pd.read_csv(
    "../data/external/datos_limpios_diagnosticos.csv", encoding="latin-1", sep=";"
)

## Obtención de pacientes ambulatorios que tuvieron una hospitalización

In [ ]:
# Obtiene los pacientes de ambulatorio, que hayan aparecido alguna vez en hospitalizados
pacientes_amb_con_hosp_por_anio = df_consultas.merge(
    df_egresos, how="left", left_on=["id_paciente", "ano"], right_on=["RUT", "ANO_EGR"]
)

# Calcula la cantidad de pacientes ambulatorios totales, y los que aparecieron en hospitalizados
# Por especialidad
conteo_de_pacientes_ambulatorios_con_hosp_por_especialidad = (
    pacientes_amb_con_hosp_por_anio.groupby(["ano", "nombre_especialidad"])
    .agg(
        n_pacientes_ambulatorios=("id_paciente", "nunique"),
        n_pacientes_que_estuvieron_en_hosp=("RUT", "nunique"),
    )
    .reset_index()
)

# Totales
conteo_de_pacientes_ambulatorios_con_hosp_totales = (
    pacientes_amb_con_hosp_por_anio.groupby(["ano"])
    .agg(
        n_pacientes_ambulatorios=("id_paciente", "nunique"),
        n_pacientes_que_estuvieron_en_hosp=("RUT", "nunique"),
    )
    .reset_index()
)

# Calcula el porcentaje de hospitalizacion de los pacientes ambulatorios
conteo_de_pacientes_ambulatorios_con_hosp_por_especialidad["fraccion_hosp"] = (
    conteo_de_pacientes_ambulatorios_con_hosp_por_especialidad["n_pacientes_que_estuvieron_en_hosp"]
    / conteo_de_pacientes_ambulatorios_con_hosp_por_especialidad["n_pacientes_ambulatorios"]
)

conteo_de_pacientes_ambulatorios_con_hosp_totales["fraccion_hosp"] = (
    conteo_de_pacientes_ambulatorios_con_hosp_totales["n_pacientes_que_estuvieron_en_hosp"]
    / conteo_de_pacientes_ambulatorios_con_hosp_totales["n_pacientes_ambulatorios"]
)

In [ ]:
# Guarda los resultados de pacientes ambulatorios
with pd.ExcelWriter("../data/interim/pacientes_ambulatorios_con_hosp.xlsx") as file:
    conteo_de_pacientes_ambulatorios_con_hosp_por_especialidad.to_excel(
        file, sheet_name="por_especialidad", index=False
    )
    conteo_de_pacientes_ambulatorios_con_hosp_totales.to_excel(
        file, sheet_name="totales", index=False
    )